# Домашнее задание №2

## Часть 1

Нужно подобрать оптимальные параметры xgboost для набора данных HR.csv в папке второго семинара. Код для проверки качества представлен в скрипте xgboost_params_checker.py, а пример набора параметров в xgboost_params_example.json. Чекер с вашими параметрами должен отработать за 2 минуты на машинке для проверки. Для сравнения на xgboost_params_example.json чекер работает 20 секунд.

## Часть 2

Нужно реализовать градиентный бустинг для задачи классификации на наборе данных HR.csv. Код для проверки качества представлен в  скрипте gb_impl_checker.py, а шаблон, от которого стоит отталкиваться, в скрипте gb_impl_example.py. Чекер вашего скрипта должен отработать за 1 минуту на машинке для проверки. Для сравнения на gb_impl_example.py чекер работает 10 секунд.

Задание сдавать в форму https://goo.gl/forms/cfzYQ3jVnNjG2k1B3

Так как нужно присылать файлы, а коды чекеров заранее выложены, то у вас будет только одна посылка.

## Выполнение задания. Часть 2

### <font color='orange'> Построим реализацию бустинга с логистической функцией потерь (log loss)</font>

In [3]:
from sklearn.model_selection import cross_val_score, GridSearchCV
import pandas as pd
data = pd.read_csv('../seminar02/HR.csv')
data[:5]
target = 'left'
features = [c for c in data if c != target]
print(features)
data.head()

['last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years']


last_evaluation  number_project  average_montly_hours  time_spend_company  \
0             0.53               2                   157                   3   
1             0.86               5                   262                   6   
2             0.88               7                   272                   4   
3             0.87               5                   223                   5   
4             0.52               2                   159                   3   

   Work_accident  left  promotion_last_5years  
0              0     1                      0  
1              0     0                      0  
2              0     1                      0  
3              0     1                      0  
4              0     1                      0

In [16]:
# Шаблон из скрипта gb_impl_example.py
#coding=utf-8

from sklearn.base import BaseEstimator
from sklearn.tree import DecisionTreeRegressor
import numpy as np


# Параметрами с которыми вы хотите обучать деревья
TREE_PARAMS_DICT = {'max_depth':19}
# Параметр tau (learning_rate) для вашего GB
TAU = 0.0001



class SimpleGB(BaseEstimator):
    def __init__(self, tree_params_dict, iters, tau):
        self.tree_params_dict = tree_params_dict
        self.iters = iters
        self.tau = tau
        self.global_res = [] # введем переменную, что бы посмотреть, что  у нас собрает переменная res
        
    def fit(self, X_data, y_data):
        self.base_algo = DecisionTreeRegressor(**self.tree_params_dict).fit(X_data, y_data)
        self.estimators = []
        curr_pred = self.base_algo.predict(X_data)
        for iter_num in range(self.iters):
            # Нужно посчитать градиент функции потерь
            grad = (curr_pred - y_data)*X_data # TODO
            # Нужно обучить DecisionTreeRegressor предсказывать антиградиент
            # Не забудьте про self.tree_params_dict
            algo = DecisionTreeRegressor(**self.tree_params_dict).fit(X_data, - grad) # TODO

            self.estimators.append(algo)
            # Обновите предсказания в каждой точке
            curr_pred += self.estimators[-1].predict(X_data) # TODO
        return self
    
    def predict(self, X_data):
        # Предсказание на данных
        res = self.base_algo.predict(X_data)
        for estimator in self.estimators:
            res += self.tau * estimator.predict(X_data)
            
        # Задача классификации, поэтому надо отдавать 0 и 1
        # Мои примечания - Нужно попробовать применить логистическую ф-ю к res для определния класса 
        t = 0.5
        res_class = res
        res_class = 1/(1+np.exp(-res)) # добавим сигмоид, что бы получить вероятности классов
        res_class[res_class > t] = 1 # если вероятность больше t=0.5, то класс 1
        res_class[res_class <= t] = 0 # в противном случае - класс 0
        
        # вариант определение класса, если предсказание больше или меньше 0
        #res_class[res_class > 0] = 1
        #res_class[res_class <=0] = 0
        return res_class


In [17]:
#my_GBM = 0
my_GBM = SimpleGB(tree_params_dict=TREE_PARAMS_DICT, iters=100, tau=0.0003)
my_GBM.fit(data[features], data[target])
my_GBM = 0
my_GBM = SimpleGB(tree_params_dict=TREE_PARAMS_DICT, iters=100, tau=0.0003)

/home/kjack/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/base.py:3140: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  return this.join(other, how=how, return_indexers=return_indexers)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
%%time
np.mean(cross_val_score(my_GBM, data[features], data[target], cv=3, scoring='accuracy'))

### Попробуем подобрать оптимальные параметры (глубина дерева и шаг)

Поскольку в чекере количество итераций зафиксировано (100 итераций), то будем перебирать только эти два параметра. Других параметров я не нашел. dir(SimpleGB) выдает только свойства и атрибуты, указанные при объявлении класса. 

In [162]:
my_params = {'max_depth': [17, 18, 19], 'tau': [0.0006, 0.0003, 0.00001], 'iter': 100}
column_list = ['mean_accuracy']
test_GBM = SimpleGB(tree_params_dict=TREE_PARAMS_DICT, iters=100, tau=0.0003)
for i in my_params.keys():
        column_list.append(i)
eval_res = pd.DataFrame(columns=column_list)
curr_res = {}
for tau in list(my_params['tau']):
    curr_res['tau'] = tau
    for depth in list(my_params['max_depth']):
        curr_res['max_depth'] = depth
        for iter in [my_params['iter']]:
            curr_res['iter'] = iter
            test_GBM = SimpleGB(tree_params_dict=TREE_PARAMS_DICT, iters=iter, tau=tau)
            curr_res['mean_accuracy'] = np.mean(cross_val_score(test_GBM, data[features], data[target], cv=3, scoring='accuracy'))
            eval_res = eval_res.append(curr_res, ignore_index=True)


In [164]:
eval_res.sort_values(by='mean_accuracy', ascending=False)

mean_accuracy  max_depth      tau   iter
0       0.720782       17.0  0.00060  100.0
5       0.720648       19.0  0.00030  100.0
8       0.720048       19.0  0.00001  100.0
7       0.719915       18.0  0.00001  100.0
3       0.719782       17.0  0.00030  100.0
2       0.718981       19.0  0.00060  100.0
4       0.718648       18.0  0.00030  100.0
6       0.718315       17.0  0.00001  100.0
1       0.718315       18.0  0.00060  100.0

In [22]:
res = my_GBM.estimators[50].predict(data[features])
for estimator in my_GBM.estimators:
    res += 0.05 * estimator.predict(data[features])

In [37]:
res_class = 1/(1+np.exp(-res))
res_class.min()

0.5396237433292548

In [38]:
res_class[res_class > 0.5] = 1
res_class[res_class <= 0.5] = 0

In [39]:
a = list(zip(res, res_class))
a[0:10]

[(0.1875727089378466, 1.0),
 (0.1811212149201806, 1.0),
 (0.19318063713300604, 1.0),
 (0.17946378535138183, 1.0),
 (0.1875727089378466, 1.0),
 (0.1875727089378466, 1.0),
 (0.19050608350254833, 1.0),
 (0.1811212149201806, 1.0),
 (0.17946378535138183, 1.0),
 (0.1875727089378466, 1.0)]

In [40]:
my_GBM.tree_params_dict

{'max_depth': 7}

In [148]:
dir(my_GBM)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_param_names',
 'base_algo',
 'estimators',
 'fit',
 'get_params',
 'iters',
 'predict',
 'set_params',
 'sigma',
 'tau',
 'tree_params_dict']